In [1]:
import os,sys,warnings,time,re,math,gc
warnings.filterwarnings("ignore")
from glob import glob
import numpy as np
import pandas as pd
import polars as pl
import polars.selectors as cs
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold,GroupKFold,train_test_split
import lightgbm as lgb
import catboost as cgb
import xgboost as xgb
import joblib

In [2]:
pl.Config.set_float_precision(2)
pl.Config.set_fmt_float("full")
pl.Config.set_tbl_rows(20)
pl.Config.set_fmt_str_lengths(200)

polars.config.Config

In [3]:
path_to_train = "/home/sohail/Downloads/credit_risk/train"
path_to_test = "/home/sohail/Downloads/credit_risk/test"

In [4]:
all_train_files = glob(path_to_train+"/*.parquet")
all_test_files = glob(path_to_test+"/*.parquet")
train_files_df = pl.DataFrame({"index":range(len(all_train_files)),"path":all_train_files})
test_files_df = pl.DataFrame({"index":range(len(all_test_files)),"path":all_test_files})
train_files_df = (
    train_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
test_files_df = (
    test_files_df
    .with_columns(
        (pl.col("path").str.split("/").list.get(-1)).alias("filename")
    )
    .sort(by="filename")
)
display(train_files_df.head())
display(test_files_df.head())

index,path,filename
i64,str,str
31,"""/home/sohail/Downloads/credit_risk/train/train_applprev_1_0.parquet""","""train_applprev_1_0.parquet"""
13,"""/home/sohail/Downloads/credit_risk/train/train_applprev_1_1.parquet""","""train_applprev_1_1.parquet"""
0,"""/home/sohail/Downloads/credit_risk/train/train_applprev_2.parquet""","""train_applprev_2.parquet"""
7,"""/home/sohail/Downloads/credit_risk/train/train_base.parquet""","""train_base.parquet"""
11,"""/home/sohail/Downloads/credit_risk/train/train_credit_bureau_a_1_0.parquet""","""train_credit_bureau_a_1_0.parquet"""


index,path,filename
i64,str,str
31,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_0.parquet""","""test_applprev_1_0.parquet"""
35,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_1.parquet""","""test_applprev_1_1.parquet"""
9,"""/home/sohail/Downloads/credit_risk/test/test_applprev_1_2.parquet""","""test_applprev_1_2.parquet"""
1,"""/home/sohail/Downloads/credit_risk/test/test_applprev_2.parquet""","""test_applprev_2.parquet"""
33,"""/home/sohail/Downloads/credit_risk/test/test_base.parquet""","""test_base.parquet"""


In [5]:
train_base = (
    pl.read_parquet(path_to_train+"/train_base.parquet")
    .select(
        pl.col("case_id").cast(pl.UInt64),
        cs.contains("date").str.to_date().alias("Date"),
        pl.col("WEEK_NUM").cast(pl.UInt8).alias("week_num"),
        pl.col("target").cast(pl.UInt8)
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.weekday().alias("weekday"),
        pl.col("Date").dt.week().alias("week"),
        (pl.col("Date").dt.year() - 2018).cast(pl.UInt8).alias("year")
    )
    .select(~cs.contains("target"),cs.contains("target"))
)
train_base

case_id,Date,week_num,month,weekday,week,year,target
u64,date,u8,i8,i8,i8,i32,u8
0,2019-01-03,0,1,4,1,1,0
1,2019-01-03,0,1,4,1,1,0
2,2019-01-04,0,1,5,1,1,0
3,2019-01-03,0,1,4,1,1,0
4,2019-01-04,0,1,5,1,1,1
5,2019-01-02,0,1,3,1,1,0
6,2019-01-03,0,1,4,1,1,0
7,2019-01-03,0,1,4,1,1,0
8,2019-01-03,0,1,4,1,1,0


In [6]:
test_base = (
    pl.read_parquet(path_to_test+"/test_base.parquet")
    .select(
        pl.col("case_id").cast(pl.UInt64),
        cs.contains("date").str.to_date().alias("Date"),
        pl.col("WEEK_NUM").cast(pl.UInt8).alias("week_num")
    )
    .with_columns(
        pl.col("Date").dt.month().alias("month"),
        pl.col("Date").dt.weekday().alias("weekday"),
        pl.col("Date").dt.week().alias("week"),
        (pl.col("Date").dt.year() - 2018).cast(pl.UInt8).alias("year"),
        pl.lit(0).cast(pl.UInt8).alias("target")
    )
    .select(~cs.contains("target"),cs.contains("target"))
)
test_base

case_id,Date,week_num,month,weekday,week,year,target
u64,date,u8,i8,i8,i8,i32,u8
57543,2020-10-06,92,10,2,41,2,0
57549,2020-10-06,92,10,2,41,2,0
57551,2020-10-06,92,10,2,41,2,0
57552,2020-10-07,92,10,3,41,2,0
57569,2020-10-06,92,10,2,41,2,0
57630,2020-10-06,92,10,2,41,2,0
57631,2020-10-06,92,10,2,41,2,0
57632,2020-10-06,92,10,2,41,2,0
57633,2020-10-06,92,10,2,41,2,0


In [7]:
def rename_cols(df:pl.DataFrame,name:str):
    if name == "intshallow":
        return df.rename(
            {
                "dpdmaxdatemonth_442T":"dpdmaxdatemonth_442D",
                "dpdmaxdatemonth_89T":"dpdmaxdatemonth_89D",
                "dpdmaxdateyear_596T":"dpdmaxdateyear_596D",
                "dpdmaxdateyear_896T":"dpdmaxdateyear_896D",
                "overdueamountmaxdatemonth_284T":"overdueamountmaxdatemonth_284D",
                "overdueamountmaxdatemonth_365T":"overdueamountmaxdatemonth_365D",
                "overdueamountmaxdateyear_2T":"overdueamountmaxdateyear_2D",
                "overdueamountmaxdateyear_994T":"overdueamountmaxdateyear_994D",
            }
        )
    elif name == "intdepth":
        return df.rename(
            {
                "pmts_month_158T":"pmts_month_158D",
                "pmts_month_706T":"pmts_month_706D",
                "pmts_year_1139T":"pmts_year_1139D",
                "pmts_year_507T":"pmts_year_507D",
            }
        )
    elif name == "extshallow":
        return df.rename(
            {
                "dpdmaxdatemonth_804T":"dpdmaxdatemonth_804D",
                "dpdmaxdateyear_742T":"dpdmaxdateyear_742D",
                "overdueamountmaxdatemonth_494T":"overdueamountmaxdatemonth_494D",
                "overdueamountmaxdateyear_432T":"overdueamountmaxdateyear_432D",
            }
        )
    else:
        return df

In [8]:
def convert_dtype(df:pl.DataFrame):
    return (
        df
        .select(
            cs.by_name("case_id").cast(pl.UInt64),
            cs.contains("num_group").cast(pl.UInt16),
            cs.ends_with("D").cast(pl.Date),
            (cs.ends_with("T","M") | (cs.ends_with("L") & cs.string())).cast(pl.String),
            (cs.ends_with("L") & cs.integer()).cast(pl.Int32),
            (cs.ends_with("L") & cs.float()).cast(pl.Float32),
            (cs.ends_with("P","A") & cs.unsigned_integer()).cast(pl.UInt32),
            (cs.ends_with("P","A") & cs.signed_integer()).cast(pl.Int32),
            (cs.ends_with("P","A") & cs.float()).cast(pl.Float32),
            pl.col(pl.Boolean)
        )
    )

In [9]:
def grouping(df:pl.DataFrame):
    return (
        df
        .group_by("case_id")
        .agg(
            cs.numeric().max().prefix("max_"),
            cs.numeric().mean().prefix("mean_"),
            cs.numeric().first().prefix("first_"),
            cs.numeric().last().prefix("last_"),
            cs.date().first().prefix("first_"),
            cs.date().last().prefix("last_"),
            (cs.string() | cs.boolean()).drop_nulls().mode().first().prefix("mode_"),
            (cs.string() | cs.boolean()).last().prefix("last_"),
            (cs.string() | cs.boolean()).first().prefix("first_")
        )
    )

In [10]:
def preprocess(
        string_name:str,
        prefix_string:str,
        train_base_df:pl.DataFrame,
        test_base_df:pl.DataFrame,
        cat_cols:list,
        num_cols:list
        ):
    train_files_list = train_files_df.filter(pl.col("filename").str.contains(string_name))["path"].to_list()
    test_files_list = test_files_df.filter(pl.col("filename").str.contains(string_name))["path"].to_list()
    with pl.StringCache():
        train_df = (
            pl.concat(
                [
                    pl.read_parquet(train_file)
                    .pipe(rename_cols,prefix_string)
                    .pipe(convert_dtype)
                    .pipe(grouping)
                    for train_file in train_files_list
                ],
                parallel=False,
                rechunk=False
            )
        )
        test_schema = {"_".join(col_name.split("_")[1:]) if col_name != "case_id" else col_name:dtype for col_name,dtype in train_df.schema.items()}
        test_columns = {col_name for col_name in test_schema.keys()}
        if prefix_string == "staticexternal":
            train_df = (
                train_df
                .with_columns(
                    cs.contains("302T").str.split("%").list.gather([0,1])
                    .map_elements(
                        lambda x: (int(x[0]) + int(x[1].strip().split(" ")[1]))/200,return_dtype=pl.Float32
                    )
                )
            )
        cat_cols += train_df.select((~(cs.numeric() | cs.date())).prefix(f"{prefix_string}_")).columns
        num_cols += train_df.select((cs.numeric() | cs.date()).exclude("case_id").prefix(f"{prefix_string}_")).columns
        train_base_df = train_base_df.join(
            train_df
            .with_columns(pl.col(pl.String).cast(pl.Categorical).rank("dense"))
            .select(pl.col("case_id"),pl.all().exclude("case_id").shrink_dtype().prefix(f"{prefix_string}_")),
            on="case_id",
            how="left"
        )
        test_df = (
            pl.concat(
                [
                    pl.read_parquet(test_file)
                    .pipe(rename_cols,prefix_string)
                    .select(test_columns)
                    .cast(test_schema)
                    .pipe(grouping)
                    for test_file in test_files_list
                ],
                parallel=False,
                rechunk=False
            )
        )
        if prefix_string == "staticexternal":
            test_df = (
                test_df
                .with_columns(
                    cs.contains("302T").str.split("%").list.gather([0,1])
                    .map_elements(
                        lambda x: (int(x[0]) + int(x[1].strip().split(" ")[1]))/200,return_dtype=pl.Float32
                    )
                )
            )
        test_base_df = test_base_df.join(
            test_df
            .with_columns(pl.col(pl.String).cast(pl.Categorical).rank("dense"))
            .select(pl.col("case_id"),pl.all().exclude("case_id").shrink_dtype().prefix(f"{prefix_string}_")),
            on="case_id",
            how="left"
        )
    return train_base_df,test_base_df,cat_cols,num_cols


In [11]:
string_list = ["applprev_1","applprev_2","static_0","static_cb","person_1","person_2","other_1","deposit_1","debitcard","bureau_a_1","bureau_a_2","bureau_b_1","bureau_b_2","registry_a","registry_b","registry_b"]
prefix_string_list = ["pastshallow","pastdepth","staticbase","staticexternal","personshallow","persondepth","othershallow","depositshallow","cardshallow","intshallow","intdepth","extshallow","extdepth","rega","regb","regc"]

In [12]:
cat_cols = []
num_cols = []
count = 0
for string_name,prefix_name in zip(string_list,prefix_string_list):
    train_base,test_base,cat_cols,num_cols = preprocess(string_name,prefix_name,train_base,test_base,cat_cols,num_cols)
    print(f"done: {count+1} for {prefix_name}")
    count += 1
    gc.collect()

done: 1 for pastshallow
done: 2 for pastdepth
done: 3 for staticbase
done: 4 for staticexternal
done: 5 for personshallow
done: 6 for persondepth
done: 7 for othershallow
done: 8 for depositshallow
done: 9 for cardshallow
done: 10 for intshallow
done: 11 for intdepth
done: 12 for extshallow
done: 13 for extdepth
done: 14 for rega
done: 15 for regb
done: 16 for regc


In [13]:
train_base = (
    train_base
    .with_columns(
        (pl.col(pl.Date) - pl.col("Date")).dt.total_days(),
        pl.col(pl.Date).exclude("Date").dt.weekday().prefix("weekdaydate_"),
        pl.col(pl.Date).exclude("Date").dt.month().prefix("monthdate_"),
        pl.col(pl.Date).exclude("Date").dt.week().prefix("weekdate_")
    )
    .drop("Date","case_id")
    .with_columns(pl.col(pl.Boolean).cast(pl.UInt8))
    .select(pl.all().shrink_dtype())
)

all_date_cols = train_base.select(cs.contains(["weekdaydate","monthdate","weekdate"])).columns

In [14]:
list_of_nan_groups = (
    train_base.select(num_cols)
    .select(pl.all().is_null().sum())
    .transpose(include_header=True)
    .group_by("column_0")
    .agg(pl.col("column"))
    .sort(by="column_0")
    ["column"].to_list()
)
len(list_of_nan_groups)

320

In [15]:
def reduce_grps(grps):
    use = []
    for g in grps:
        mx = 0; vx = g[0]
        for gg in g:
            n = train_base.select(gg).n_unique()
            if n > mx:
                mx = n
                vx = gg
        use.append(vx)
    return use

def group_col_by_corr(mat:pl.DataFrame,thresh=0.8):
    remaining_cols = mat.columns
    groups = []
    while remaining_cols:
        col_name = remaining_cols.pop(0)
        corr_cols = (
        mat
        .fill_null(0)
        .corr()
        .pipe(lambda df: df.select(pl.Series(name="col_name",values=df.columns),pl.all()))
        .select("col_name",col_name)
        .filter(pl.col(col_name) >= thresh)
        ["col_name"].to_list()
    )
        groups.append(corr_cols)
        remaining_cols = [c for c in remaining_cols if c not in corr_cols]

    return groups


uses = []
count = 0
for k in list_of_nan_groups[1:]:
    if len(k) > 1:
        grps = group_col_by_corr(train_base.select(k))
        use = reduce_grps(grps)
        uses += use
    else:
        uses += k
    count += 1

len(uses)

486

In [16]:
sorted(list_of_nan_groups[0])

['personshallow_first_birth_259D',
 'personshallow_first_mainoccupationinc_384A',
 'personshallow_first_num_group1',
 'personshallow_first_personindex_1023L',
 'personshallow_first_persontype_1072L',
 'personshallow_first_persontype_792L',
 'personshallow_last_num_group1',
 'personshallow_max_mainoccupationinc_384A',
 'personshallow_max_num_group1',
 'personshallow_max_personindex_1023L',
 'personshallow_max_persontype_1072L',
 'personshallow_max_persontype_792L',
 'personshallow_mean_mainoccupationinc_384A',
 'personshallow_mean_num_group1',
 'personshallow_mean_personindex_1023L',
 'personshallow_mean_persontype_1072L',
 'personshallow_mean_persontype_792L',
 'staticbase_first_annuity_780A',
 'staticbase_first_applicationcnt_361L',
 'staticbase_first_applications30d_658L',
 'staticbase_first_applicationscnt_1086L',
 'staticbase_first_applicationscnt_464L',
 'staticbase_first_applicationscnt_629L',
 'staticbase_first_applicationscnt_867L',
 'staticbase_first_clientscnt12m_3712952L',
 

In [26]:
len(uses)

486

In [27]:
len(set(uses))

481

In [29]:
train_base = train_base.select(list_of_nan_groups[0]+list(set(uses))+cat_cols)
train_base.head()

staticbase_max_applicationcnt_361L,staticbase_max_applications30d_658L,staticbase_max_applicationscnt_1086L,staticbase_max_applicationscnt_464L,staticbase_max_applicationscnt_629L,staticbase_max_applicationscnt_867L,staticbase_max_clientscnt12m_3712952L,staticbase_max_clientscnt3m_3712950L,staticbase_max_clientscnt6m_3712949L,staticbase_max_clientscnt_100L,staticbase_max_clientscnt_1022L,staticbase_max_clientscnt_1071L,staticbase_max_clientscnt_1130L,staticbase_max_clientscnt_157L,staticbase_max_clientscnt_257L,staticbase_max_clientscnt_304L,staticbase_max_clientscnt_360L,staticbase_max_clientscnt_493L,staticbase_max_clientscnt_533L,staticbase_max_clientscnt_887L,staticbase_max_clientscnt_946L,staticbase_max_deferredmnthsnum_166L,staticbase_max_homephncnt_628L,staticbase_max_mobilephncnt_593L,staticbase_max_numactivecreds_622L,staticbase_max_numactivecredschannel_414L,staticbase_max_numactiverelcontr_750L,staticbase_max_numcontrs3months_479L,staticbase_max_numnotactivated_1143L,staticbase_max_numpmtchanneldd_318L,staticbase_max_numrejects9m_859L,staticbase_max_sellerplacecnt_915L,staticbase_max_sellerplacescnt_216L,staticbase_max_annuity_780A,staticbase_max_credamount_770A,staticbase_max_disbursedcredamount_1113A,staticbase_max_downpmt_116A,…,extshallow_mode_contracttype_653M,extshallow_mode_credor_3940957M,extshallow_mode_periodicityofpmts_997M,extshallow_mode_pmtmethod_731M,extshallow_mode_purposeofcred_722M,extshallow_mode_subjectrole_326M,extshallow_mode_subjectrole_43M,extshallow_mode_periodicityofpmts_997L,extshallow_last_classificationofcontr_1114M,extshallow_last_contractst_516M,extshallow_last_contracttype_653M,extshallow_last_credor_3940957M,extshallow_last_periodicityofpmts_997M,extshallow_last_pmtmethod_731M,extshallow_last_purposeofcred_722M,extshallow_last_subjectrole_326M,extshallow_last_subjectrole_43M,extshallow_last_periodicityofpmts_997L,extshallow_first_classificationofcontr_1114M,extshallow_first_contractst_516M,extshallow_first_contracttype_653M,extshallow_first_credor_3940957M,extshallow_first_periodicityofpmts_997M,extshallow_first_pmtmethod_731M,extshallow_first_purposeofcred_722M,extshallow_first_subjectrole_326M,extshallow_first_subjectrole_43M,extshallow_first_periodicityofpmts_997L,rega_mode_name_4527232M,rega_last_name_4527232M,rega_first_name_4527232M,regb_mode_name_4917606M,regb_last_name_4917606M,regb_first_name_4917606M,regc_mode_name_4917606M,regc_last_name_4917606M,regc_first_name_4917606M
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u32,u32,u32,u16,u16,u16,u16,u16,u16
0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1917.60,30000.00,30000.00,0.00,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3134.00,19999.80,19999.80,0.00,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,2.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,4937.00,78000.00,78000.00,0.00,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
0.00,1.00,0.00,2.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.